# Processing the data individually

# Trial After Limits Raising

In [ ]:
from openai import OpenAI
# create an openAI connection
client = OpenAI()

In [44]:
# upload the batch file to the server
batch_input_file = client.files.create(
    file=open("./data.jsonl", "rb"),
    purpose="batch"
)

print(batch_input_file)

FileObject(id='file-9aLij28eopt5P9vRXaM9LW', bytes=58107791, created_at=1737070835, filename='data.jsonl', object='file', purpose='batch', status='processed', status_details=None)


In [45]:
# create a batch work at the server
batch_input_file_id = batch_input_file.id
batch_info = client.batches.create(
    input_file_id=batch_input_file_id,
    endpoint="/v1/chat/completions",
    completion_window="24h",
    metadata={
        "description": "ner labelling"
    }
)

# print the batch information
print(batch_info)

Batch(id='batch_67899950d9ac8190861d53e801b2d552', completion_window='24h', created_at=1737070928, endpoint='/v1/chat/completions', input_file_id='file-9aLij28eopt5P9vRXaM9LW', object='batch', status='validating', cancelled_at=None, cancelling_at=None, completed_at=None, error_file_id=None, errors=None, expired_at=None, expires_at=1737157328, failed_at=None, finalizing_at=None, in_progress_at=None, metadata={'description': 'ner labelling'}, output_file_id=None, request_counts=BatchRequestCounts(completed=0, failed=0, total=0))


In [52]:
# check on the status of the batch
batch_status = client.batches.retrieve(batch_info.id)
print(batch_status)

Batch(id='batch_67899950d9ac8190861d53e801b2d552', completion_window='24h', created_at=1737070928, endpoint='/v1/chat/completions', input_file_id='file-9aLij28eopt5P9vRXaM9LW', object='batch', status='completed', cancelled_at=None, cancelling_at=None, completed_at=1737073360, error_file_id=None, errors=None, expired_at=None, expires_at=1737157328, failed_at=None, finalizing_at=1737072613, in_progress_at=1737070931, metadata={'description': 'ner labelling'}, output_file_id='file-42QNSZqMV1z3Zi7NwDEq9h', request_counts=BatchRequestCounts(completed=7310, failed=0, total=7310))


In [57]:
output_file_id = "file-42QNSZqMV1z3Zi7NwDEq9h"

In [58]:
# code to retrieve the batch results 
file_response = client.files.content("file-42QNSZqMV1z3Zi7NwDEq9h")

In [59]:
# show a list of all the batches
client.batches.list(limit=10)

SyncCursorPage[Batch](data=[Batch(id='batch_67899950d9ac8190861d53e801b2d552', completion_window='24h', created_at=1737070928, endpoint='/v1/chat/completions', input_file_id='file-9aLij28eopt5P9vRXaM9LW', object='batch', status='completed', cancelled_at=None, cancelling_at=None, completed_at=1737073360, error_file_id=None, errors=None, expired_at=None, expires_at=1737157328, failed_at=None, finalizing_at=1737072613, in_progress_at=1737070931, metadata={'description': 'ner labelling'}, output_file_id='file-42QNSZqMV1z3Zi7NwDEq9h', request_counts=BatchRequestCounts(completed=7310, failed=0, total=7310)), Batch(id='batch_6789887331f88190941bb3234d57687e', completion_window='24h', created_at=1737066611, endpoint='/v1/chat/completions', input_file_id='file-Smkf2fgapcyuDJrQaos43Y', object='batch', status='failed', cancelled_at=None, cancelling_at=None, completed_at=None, error_file_id=None, errors=Errors(data=[BatchError(code='token_limit_exceeded', line=None, message='Enqueued token limit r

In [60]:
with open("batch_output.jsonl", "w") as f:
    f.write(file_response.text) 

# Exploring data

In [28]:
import pandas as pd
import json

In [29]:
# Reading the data
import jsonlines

data = []
with jsonlines.open('batch_output.jsonl') as reader:
    for obj in reader:
        data.append(obj)

In [30]:
data[0]

{'id': 'batch_req_67899fe58c8c8190a0955ad225195809',
 'custom_id': 'request-0',
 'response': {'status_code': 200,
  'request_id': '99ab28938ec092d4cddac493514ec1bc',
  'body': {'id': 'chatcmpl-AqTn6aza0A2i0z2O4DhTlzVgCvthV',
   'object': 'chat.completion',
   'created': 1737070936,
   'model': 'gpt-4o-mini-2024-07-18',
   'choices': [{'index': 0,
     'message': {'role': 'assistant',
      'content': '```json\n{\n  "entities": [\n    {\n      "text": "د. محمود السمرة",\n      "category": "PERSON",\n      "start_position": 224,\n      "end_position": 241\n    },\n    {\n      "text": "د. أمل نصير",\n      "category": "PERSON",\n      "start_position": 292,\n      "end_position": 304\n    },\n    {\n      "text": "طه حسين",\n      "category": "PERSON",\n      "start_position": 363,\n      "end_position": 371\n    },\n    {\n      "text": "الأربعاء",\n      "category": "ORDINAL",\n      "start_position": 373,\n      "end_position": 380\n    },\n    {\n      "text": "صحيفة الغد",\n      "c

In [31]:
extracted_data = []

for entry in data:
    try:
        choices = entry.get('response', {}).get('body', {}).get('choices', [])
        if choices:
            message_content = choices[0].get('message', {}).get('content', '')
        else:
            message_content = ''
        
        extracted_entry = {
            'custom_id': entry.get('custom_id', ''),
            'content': message_content
        }
        extracted_data.append(extracted_entry)
    except KeyError as e:
        print(f"Key error: {e}")
    except Exception as e:
        print(f"An error occurred: {e}")


In [32]:
output_df = pd.DataFrame(extracted_data)
output_df.rename(columns={"content":"entities"}, inplace = True)
output_df

,custom_id,entities
0,request-0,"```json\n{\n ""entities"": [\n {\n ""tex..."
1,request-1,"```json\n{\n ""entities"": [\n {\n ""tex..."
2,request-2,"```json\n{\n ""entities"": [\n {\n ""tex..."
3,request-3,"```json\n{\n ""entities"": [\n {\n ""tex..."
4,request-4,"```json\n{\n ""entities"": [\n {\n ""tex..."
...,...,...
7305,request-7305,"```json\n{\n ""entities"": [\n {\n ""tex..."
7306,request-7306,"```json\n{\n ""entities"": [\n {\n ""tex..."
7307,request-7307,"```json\n{\n ""entities"": [\n {\n ""tex..."
7308,request-7308,"```json\n{\n ""entities"": [\n {\n ""tex..."


In [33]:
# reading main data
original_data = []
with jsonlines.open('data.jsonl') as reader:
    for obj in reader:
        original_data.append(obj)

In [34]:
data_dic = []

for entry in original_data:
    try:
        text = entry.get("body",{}).get("messages", {})[1].get("content", {}) 
        extracted_entry = {
            'custom_id': entry.get('custom_id', ''),
            'text': text
        }
        data_dic.append(extracted_entry)
    except KeyError as e:
        print(f"Key error: {e}")
    except Exception as e:
        print(f"An error occurred: {e}")

In [36]:
data_df = pd.DataFrame(data_dic)
data_df

,custom_id,text
0,request-0,يتناول المهرجان هذا العام العلاقة بين الشعر وا...
1,request-1,لاشكّ فى أنّنى لا أرمى من وراء هذا العنوان إلى...
2,request-2,لفت الدكتور خالد جودة إلى الإتجاهات المستحدثة ...
3,request-3,لفت الدكتور خالد جودة إلى الإتجاهات المستحدثة ...
4,request-4,أصدر فاروق حسنى وزير الثقافة المصري قرارا بتعي...
...,...,...
7305,request-7305,موقع الكون - المجموعة الشمسية - أقمار زحل عودة...
7306,request-7306,موقع الكون - المجموعة الشمسية - أقمار زحل عودة...
7307,request-7307,موقع الكون - المجموعة الشمسية - أقمار زحل عودة...
7308,request-7308,موقع الكون - المجموعة الشمسية - حلقات زحل عودة...


In [37]:
# Merge the two dataframes on their custom-id
df = pd.merge(data_df, output_df, on="custom_id")
df

,custom_id,text,entities
0,request-0,يتناول المهرجان هذا العام العلاقة بين الشعر وا...,"```json\n{\n ""entities"": [\n {\n ""tex..."
1,request-1,لاشكّ فى أنّنى لا أرمى من وراء هذا العنوان إلى...,"```json\n{\n ""entities"": [\n {\n ""tex..."
2,request-2,لفت الدكتور خالد جودة إلى الإتجاهات المستحدثة ...,"```json\n{\n ""entities"": [\n {\n ""tex..."
3,request-3,لفت الدكتور خالد جودة إلى الإتجاهات المستحدثة ...,"```json\n{\n ""entities"": [\n {\n ""tex..."
4,request-4,أصدر فاروق حسنى وزير الثقافة المصري قرارا بتعي...,"```json\n{\n ""entities"": [\n {\n ""tex..."
...,...,...,...
7305,request-7305,موقع الكون - المجموعة الشمسية - أقمار زحل عودة...,"```json\n{\n ""entities"": [\n {\n ""tex..."
7306,request-7306,موقع الكون - المجموعة الشمسية - أقمار زحل عودة...,"```json\n{\n ""entities"": [\n {\n ""tex..."
7307,request-7307,موقع الكون - المجموعة الشمسية - أقمار زحل عودة...,"```json\n{\n ""entities"": [\n {\n ""tex..."
7308,request-7308,موقع الكون - المجموعة الشمسية - حلقات زحل عودة...,"```json\n{\n ""entities"": [\n {\n ""tex..."


In [38]:
# obtaining the title from the
raw_data = pd.read_csv("data.csv")
raw_data

,title,text
0,الأدب العربي-أدبيات,يتناول المهرجان هذا العام العلاقة بين الشعر وا...
1,الأدب العربي-أدبيات,لاشكّ فى أنّنى لا أرمى من وراء هذا العنوان إلى...
2,الأدب العربي-أدبيات,لفت الدكتور خالد جودة إلى الإتجاهات المستحدثة ...
3,الأدب العربي-أدبيات,لفت الدكتور خالد جودة إلى الإتجاهات المستحدثة ...
4,الأدب العربي-أدبيات,أصدر فاروق حسنى وزير الثقافة المصري قرارا بتعي...
...,...,...
7305,فلك-علوم بحتة,موقع الكون - المجموعة الشمسية - أقمار زحل عودة...
7306,فلك-علوم بحتة,موقع الكون - المجموعة الشمسية - أقمار زحل عودة...
7307,فلك-علوم بحتة,موقع الكون - المجموعة الشمسية - أقمار زحل عودة...
7308,فلك-علوم بحتة,موقع الكون - المجموعة الشمسية - حلقات زحل عودة...


In [39]:
# merging the title in 
df = pd.merge(df, raw_data, left_index=True, right_index=True)
df.head(2)


,custom_id,text_x,entities,title,text_y
0,request-0,يتناول المهرجان هذا العام العلاقة بين الشعر وا...,"```json\n{\n ""entities"": [\n {\n ""tex...",الأدب العربي-أدبيات,يتناول المهرجان هذا العام العلاقة بين الشعر وا...
1,request-1,لاشكّ فى أنّنى لا أرمى من وراء هذا العنوان إلى...,"```json\n{\n ""entities"": [\n {\n ""tex...",الأدب العربي-أدبيات,لاشكّ فى أنّنى لا أرمى من وراء هذا العنوان إلى...


In [40]:
df.drop(columns=["text_y"], inplace=True)
df.rename(columns={"text_x":"text"}, inplace=True)
df.head(2)

,custom_id,text,entities,title
0,request-0,يتناول المهرجان هذا العام العلاقة بين الشعر وا...,"```json\n{\n ""entities"": [\n {\n ""tex...",الأدب العربي-أدبيات
1,request-1,لاشكّ فى أنّنى لا أرمى من وراء هذا العنوان إلى...,"```json\n{\n ""entities"": [\n {\n ""tex...",الأدب العربي-أدبيات


In [41]:
# Store the final combined Dataset
df.to_csv("labelled_data.csv", encoding="utf-8", index=False)

# Experimenting with Data

In [3]:
# Import the data
import pandas as pd
import json
import urllib.parse
df = pd.read_csv("labelled_data.csv", encoding='utf-8-sig')
df.sample(3)

,custom_id,text,entities,title
2659,request-2659,موقع القانون الليبي - Law Of Libya - المقالات ...,"```json\n{\n ""entities"": [\n {\n ""tex...",القانون-علوم اجتماعية
3053,request-3053,موقع القانون الليبي - Law Of Libya - المرأة ال...,"```json\n{\n ""entities"": [\n {\n ""tex...",القانون-علوم اجتماعية
2474,request-2474,صدر عن الشيخ محمد بن مبارك آل خليفة نائب رئيس ...,"```json\n{\n ""entities"": [\n {\n ""tex...",القانون-علوم اجتماعية


In [4]:
# Converting the text into BIO tagging
def apply_bio_tagging(row):
    try: 
        text = row.text
        # entities = urllib.parse.unquote(row["entities"].strip("```").replace("json", "",1))
        entities = json.loads(row["entities"].strip("```").replace("json", "",1))["entities"]
        # entities = json.loads(row['entities']).strip("```")['entities']
        
        # tokenizing the text into words and making them all tagged as O
        text_tokens = text.split()  
        bio_tags = ['O'] * len(text_tokens)  
        
        # Iterate over the entities and mark BIO tags
        for entity in entities:
            entity_text = entity['text']
            entity_category = entity['category']

            # Find the tokens that belong to the entity
            entity_tokens = entity_text.split() 
            
            # BO tagging the first token of the entity as 'B-{category}' and others as 'I-{category}'
            
            for idx, entity_token in enumerate(entity_tokens):
                try:
                    if idx == 0:
                        bio_tags[text_tokens.index(entity_token)] = f'B-{entity_category}'
                    else:
                        bio_tags[text_tokens.index(entity_token)] = f'I-{entity_category}'
                except:
                    # get the index of the text token that contains the entity-token and then mark it
                    matching_index = next((index for index, item in enumerate(text_tokens) if entity_token in item), None)
                    if matching_index is not None:
                        if idx == 0:
                            bio_tags[matching_index] = f'B-{entity_category}'
                        else:
                            bio_tags[matching_index] = f'I-{entity_category}'   

    except:
        print(row)
        pass
    return bio_tags



In [5]:
# Apply the function to the dataframe
df['bio_tags'] = df.apply(apply_bio_tagging, axis=1)

# Display the resulta
df.head()

custom_id                                          request-880
text         CNNArabic.com - السعودية تقود موجة مكاسب بالأس...
entities     ```json\n{\n  "entities": [\n    {\n      "tex...
title                                 الإقتصاد-علوم إجتماعية
Name: 880, dtype: object


UnboundLocalError: local variable 'bio_tags' referenced before assignment

In [ ]:
apply_bio_tagging(df.iloc[880])

In [29]:
df.entities[0]

'```json\n{\n  "entities": [\n    {\n      "text": "د. محمود السمرة",\n      "category": "PERSON",\n      "start_position": 224,\n      "end_position": 241\n    },\n    {\n      "text": "د. أمل نصير",\n      "category": "PERSON",\n      "start_position": 292,\n      "end_position": 304\n    },\n    {\n      "text": "طه حسين",\n      "category": "PERSON",\n      "start_position": 363,\n      "end_position": 371\n    },\n    {\n      "text": "الأربعاء",\n      "category": "ORDINAL",\n      "start_position": 373,\n      "end_position": 380\n    },\n    {\n      "text": "صحيفة الغد",\n      "category": "ORG",\n      "start_position": 524,\n      "end_position": 537\n    },\n    {\n      "text": "العالم",\n      "category": "LOC",\n      "start_position": 22,\n      "end_position": 27\n    },\n    {\n      "text": "العقاد",\n      "category": "PERSON",\n      "start_position": 812,\n      "end_position": 818\n    },\n    {\n      "text": "الكافوريات",\n      "category": "CARDINAL",\n      "

In [32]:
import json
import urllib.parse

# The input string (URL-encoded JSON)
json_str = '''```json\n{\n  "entities": [\n    {\n      "text": "د. محمود السمرة",\n      "category": "PERSON",\n      "start_position": 224,\n      "end_position": 241\n    },\n    {\n      "text": "د. أمل نصير",\n      "category": "PERSON",\n      "start_position": 292,\n      "end_position": 304\n    },\n    {\n      "text": "طه حسين",\n      "category": "PERSON",\n      "start_position": 363,\n      "end_position": 371\n    },\n    {\n      "text": "الأربعاء",\n      "category": "ORDINAL",\n      "start_position": 373,\n      "end_position": 380\n    },\n    {\n      "text": "صحيفة الغد",\n      "category": "ORG",\n      "start_position": 524,\n      "end_position": 537\n    },\n    {\n      "text": "العالم",\n      "category": "LOC",\n      "start_position": 22,\n      "end_position": 27\n    },\n    {\n      "text": "العقاد",\n      "category": "PERSON",\n      "start_position": 812,\n      "end_position": 818\n    },\n    {\n      "text": "الكافوريات",\n      "category": "CARDINAL",\n      "start_position": 815,\n      "end_position": 828\n    },\n    {\n      "text": "الزمن",\n      "category": "CARDINAL",\n      "start_position": 996,\n      "end_position": 1001\n    }\n  ]\n}\n```'''

# Decode the URL-encoded text
decoded_str = urllib.parse.unquote(json_str.strip("```").replace("json", "",1))

# Parse the decoded string into a Python dictionary (JSON)
data = json.loads(json_str.strip("```").replace("json", "",1))

# Print the JSON object (with non-ASCII characters properly displayed)
print(json.dumps(data, ensure_ascii=False, indent=2))


{
  "entities": [
    {
      "text": "د. محمود السمرة",
      "category": "PERSON",
      "start_position": 224,
      "end_position": 241
    },
    {
      "text": "د. أمل نصير",
      "category": "PERSON",
      "start_position": 292,
      "end_position": 304
    },
    {
      "text": "طه حسين",
      "category": "PERSON",
      "start_position": 363,
      "end_position": 371
    },
    {
      "text": "الأربعاء",
      "category": "ORDINAL",
      "start_position": 373,
      "end_position": 380
    },
    {
      "text": "صحيفة الغد",
      "category": "ORG",
      "start_position": 524,
      "end_position": 537
    },
    {
      "text": "العالم",
      "category": "LOC",
      "start_position": 22,
      "end_position": 27
    },
    {
      "text": "العقاد",
      "category": "PERSON",
      "start_position": 812,
      "end_position": 818
    },
    {
      "text": "الكافوريات",
      "category": "CARDINAL",
      "start_position": 815,
      "end_position": 828
    },
  

In [14]:
import numpy as np
import pandas as pd
df = pd.read_csv("data.csv")

In [3]:
df.shape

(7310, 2)

In [20]:
df.head(2)

,title,text
0,الأدب العربي-أدبيات,يتناول المهرجان هذا العام العلاقة بين الشعر وا...
1,الأدب العربي-أدبيات,لاشكّ فى أنّنى لا أرمى من وراء هذا العنوان إلى...


In [10]:
df["title"].unique()

array(['الأدب العربي-أدبيات', 'الإقتصاد-علوم إجتماعية',
       'السياسة-علوم اجتماعية', 'القانون-علوم اجتماعية', 'رياضة',
       'عام- فنون', 'عام-إسلام-ديانات', 'علم الكمبيوتر-علوم بحتة',
       'علوم صحية-علوم تطبيقية', 'فلك-علوم بحتة'], dtype=object)

In [21]:
uni_val = df["text"].unique()

In [22]:
uni_val

array(['يتناول المهرجان هذا العام العلاقة بين الشعر والجنون والحزن. ويتضمن أمسيات شعرية مع عروض إخراجية للقصائد بمصاحبة الموسيقى والترجمات للهولندية والإنكليزية، إضافة إلى عروض سينمائية وموسيقية، ومحاضرات وجلسات حوارية تتناول هذه الموضوعة. ويسلط مهرجان هذا العام الضوء لأول مرة على الشعر في منطقة القوقاز. تبدأ الفعاليات السبت 16 حزيران / مايو بقصيدة "الكلمة الواحدة" وهي عبارة عن ملحمة غنائية طويلة يطلقها جميع الشعراء المشاركين في موكب ضخم، حيث يقرأ كل شاعر قصيدة تتكون من كلمة واحدة فقط تمثل مُلهما رئيسيًّا له، أو كلمة يراها ضائعة من الزمن والقاموس، أو كلمة تعد مفتاحا أو رمزا لشيء محبب بالنسبة له. تيمة المهرجان هي العلاقة ما بين الشعر والجنون، أو الخيط الرهيف الفاصل بين الإبداع وبين الاختلال العقلي والجنون، والدور الذي يلعبه الحزن في تعزيز كلٍّ منهما. يقول د. محمود السمرة في تقديمه لكتاب د. أمل نصير " حول نار الشعر القديم: مقاربات نقدية " الصادر حديثا " هذه الدراسة قيمة، تقدمها لنا الدكتورة امل نصير، فإذا بالشعر العصي على الفهم، شعر جميل واضح، تقف عنده معجبا بما تقرأ وتسأل نفسك: كيف فاتك

In [24]:
labelled_data = pd.read_csv("labelled_data.csv", encoding="utf-8")

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xa1 in position 0: invalid start byte

In [46]:
print ("""content":"Identify and label named entities in the given text block using a predefined list of named entity recognition (NER) categories. Ensure the response is structured in a JSON format.\n\n# Steps\n\n1. **Input Analysis**: Review the provided text block.\n2. **Entity Recognition**: Identify named entities in the text based on the predefined NER list.\n3. **Entity Labeling**: Assign the appropriate NER category to each identified entity.\n4. **Structure Output**: Organize the labeled entities into the specified JSON format.\n\n# Output Format\n\nThe output should be in the following JSON format:\n```json\n{\n  \"entities\": [\n    {\n      \"text\": \"[entity_text]\",\n      \"category\": \"[ner_category]\",\n      \"start_position\": [start_index],\n      \"end_position\": [end_index]\n    },\n    ...\n  ]\n}\n```\n# NER List\n\nPERSON: أسماء الأفراد (على سبيل المثال، \"أحمد محمد\").\nGPE (Geo-Political Entity): أسماء الدول، المدن، أو الولايات (على سبيل المثال، \"مصر\"، \"الرياض\").\nORG: أسماء الشركات، الوكالات، أو المؤسسات (على سبيل المثال، \"الأمم المتحدة\"، \"شركة أرامكو\").\nLOC: المواقع غير الجغرافية السياسية، مثل الجبال أو الأنهار (على سبيل المثال، \"جبال الهيمالايا\"، \"نهر النيل\").\nDATE: التواريخ الموجودة في النص (على سبيل المثال، \"1 يناير 2025\")\nTIME: أوقات محددة (على سبيل المثال، \"الساعة 2:00 مساءً\").\nMONEY: القيم المالية (على سبيل المثال، \"$20\"، \"15 ريال\").\nPERCENT: النسب المئوية (على سبيل المثال، \"50%\").\nQUANTITY: الكميات القابلة للقياس (على سبيل المثال، \"5 كيلومترات\"، \"20 كيلوجرام\").\nORDINAL: الترتيب أو التصنيف (على سبيل المثال، \"الأول\"، \"الثاني\").\nCARDINAL: الأرقام التي لا تندرج تحت فئات أخرى (على سبيل المثال، \"اثنان\"، \"100\")\n\n\n# Example\n\n**Input**: \n\"The CEO of OpenAI, Sam Altman, gave a talk in San Francisco on artificial intelligence.\"\n\n**Output**: \n```json\n{\n  \"entities\": [\n    {\n      \"text\": \"OpenAI\",\n      \"category\": \"ORG\",\n      \"start_position\": 11,\n      \"end_position\": 17\n    },\n    {\n      \"text\": \"Sam Altman\",\n      \"category\": \"PERSON\",\n      \"start_position\": 23,\n      \"end_position\": 33\n    },\n    {\n      \"text\": \"San Francisco\",\n      \"category\": \"GPE\",\n      \"start_position\": 52,\n      \"end_position\": 65\n    }\n  ]\n}\n```\n\n# Notes\n\n- Ensure all entities are correctly identified and labeled according to the predefined NER list.\n- Consider edge cases where entities may overlap or have multiple possible classifications.\n- Be precise in determining the start and end positions of each entity within the text.
       """)

content":"Identify and label named entities in the given text block using a predefined list of named entity recognition (NER) categories. Ensure the response is structured in a JSON format.

# Steps

1. **Input Analysis**: Review the provided text block.
2. **Entity Recognition**: Identify named entities in the text based on the predefined NER list.
3. **Entity Labeling**: Assign the appropriate NER category to each identified entity.
4. **Structure Output**: Organize the labeled entities into the specified JSON format.

# Output Format

The output should be in the following JSON format:
```json
{
  "entities": [
    {
      "text": "[entity_text]",
      "category": "[ner_category]",
      "start_position": [start_index],
      "end_position": [end_index]
    },
    ...
  ]
}
```
# NER List

PERSON: أسماء الأفراد (على سبيل المثال، "أحمد محمد").
GPE (Geo-Political Entity): أسماء الدول، المدن، أو الولايات (على سبيل المثال، "مصر"، "الرياض").
ORG: أسماء الشركات، الوكالات، أو المؤسسات (على